In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mesothelioma"
cohort = "GSE131027"

# Input paths
in_trait_dir = "../../input/GEO/Mesothelioma"
in_cohort_dir = "../../input/GEO/Mesothelioma/GSE131027"

# Output paths
out_data_file = "../../output/preprocess/Mesothelioma/GSE131027.csv"
out_gene_data_file = "../../output/preprocess/Mesothelioma/gene_data/GSE131027.csv"
out_clinical_data_file = "../../output/preprocess/Mesothelioma/clinical_data/GSE131027.csv"
json_path = "../../output/preprocess/Mesothelioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE131027_family.soft.gz', 'GSE131027_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mesothelioma/GSE131027/GSE131027_family.soft.gz
Matrix file: ../../input/GEO/Mesothelioma/GSE131027/GSE131027_series_matrix.txt.gz


Background Information:
!Series_title	"High frequency of pathogenic germline variants in genes associated with homologous recombination repair in patients with advanced solid cancers"
!Series_summary	"We identified pathogenic and likely pathogenic variants in 17.8% of the patients within a wide range of cancer types. In particular, mesothelioma, ovarian cancer, cervical cancer, urothelial cancer, and cancer of unknown primary origin displayed high frequencies of pathogenic variants. In total, 22 BRCA1 and BRCA2 germline variant were identified in 12 different cancer types, of which 10 (45%) variants were not previously identified in these patients. Pathogenic germline variants were predominantly found in DNA repair pathways; approximately half of the variants were within genes involved in homologous recombination repair. Loss of heterozygosity and somatic second hits were identified in several of these genes, supporting possible causality for cancer development. A potential treatment t

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import re
import gzip
import io
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# This dataset seems to focus on germline mutations in cancer patients
# The series matrix likely contains gene expression data based on the Series_summary and Series_overall_design
is_gene_available = True  # Gene expression data appears to be available

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Mesothelioma), check if it's in the sample characteristics
trait_row = 1  # 'cancer' is in row 1, which contains various cancer types including Mesothelioma

# No explicit age information is available in the sample characteristics
age_row = None

# No explicit gender information is available
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> Optional[int]:
    """Convert cancer type to binary indicating if it's Mesothelioma or not."""
    if value is None:
        return None
    
    # Extract the value after the colon if it exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if the value is Mesothelioma (case-insensitive)
    if re.search(r'mesothelioma', value, re.IGNORECASE):
        return 1
    else:
        return 0

def convert_age(value: str) -> Optional[float]:
    """Convert age to float."""
    # Not needed as age data isn't available, but included for completeness
    if value is None:
        return None
    
    # Extract the value after the colon if it exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (0 for female, 1 for male)."""
    # Not needed as gender data isn't available, but included for completeness
    if value is None:
        return None
    
    # Extract the value after the colon if it exists
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if re.search(r'female|f', value, re.IGNORECASE):
        return 0
    elif re.search(r'male|m', value, re.IGNORECASE):
        return 1
    else:
        return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Initial filtering and saving metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Read the gzipped file with proper decompression
    with gzip.open(os.path.join(in_cohort_dir, "GSE131027_series_matrix.txt.gz"), 'rt') as f:
        lines = f.readlines()
    
    # Extract sample characteristics lines
    sample_char_lines = [line.strip() for line in lines if line.startswith('!Sample_characteristics_ch1')]
    
    # Extract sample IDs
    sample_ids = None
    for line in lines:
        if line.startswith('!Sample_geo_accession'):
            sample_ids = line.strip().split('\t')[1:]
            break
    
    # Create a DataFrame from the sample characteristics
    data = []
    for line in sample_char_lines:
        parts = line.strip().split('\t')
        # Skip the header part
        row_data = parts[1:]
        data.append(row_data)
    
    # Convert to DataFrame
    clinical_data = pd.DataFrame(data, columns=sample_ids)
    
    # Use the library function to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save the clinical data to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'"GSM3759992"': [0.0], '"GSM3759993"': [0.0], '"GSM3759994"': [0.0], '"GSM3759995"': [0.0], '"GSM3759996"': [1.0], '"GSM3759997"': [0.0], '"GSM3759998"': [0.0], '"GSM3759999"': [0.0], '"GSM3760000"': [0.0], '"GSM3760001"': [0.0], '"GSM3760002"': [0.0], '"GSM3760003"': [0.0], '"GSM3760004"': [0.0], '"GSM3760005"': [1.0], '"GSM3760006"': [0.0], '"GSM3760007"': [1.0], '"GSM3760008"': [0.0], '"GSM3760009"': [0.0], '"GSM3760010"': [0.0], '"GSM3760011"': [0.0], '"GSM3760012"': [0.0], '"GSM3760013"': [0.0], '"GSM3760014"': [0.0], '"GSM3760015"': [0.0], '"GSM3760016"': [0.0], '"GSM3760017"': [0.0], '"GSM3760018"': [0.0], '"GSM3760019"': [0.0], '"GSM3760020"': [0.0], '"GSM3760021"': [0.0], '"GSM3760022"': [0.0], '"GSM3760023"': [1.0], '"GSM3760024"': [0.0], '"GSM3760025"': [0.0], '"GSM3760026"': [0.0], '"GSM3760027"': [0.0], '"GSM3760028"': [0.0], '"GSM3760029"': [0.0], '"GSM3760030"': [0.0], '"GSM3760031"': [0.0], '"GSM3760032"': [0.0], '"GSM3760033"': 

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 54675
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, they appear to be Affymetrix probe IDs (like "1007_s_at", "1053_at")
# rather than standard human gene symbols (which would look like "BRCA1", "TP53", etc.)
# Affymetrix probe IDs need to be mapped to human gene symbols for better interpretability

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Extract gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)
print(f"Extracted raw gene expression data: {gene_data.shape[0]} probes × {gene_data.shape[1]} samples")

# 2. Determine which columns to use for mapping
# Looking at the gene annotation dictionary, we need:
# - 'ID' column: contains the probe IDs that match the gene expression data (e.g., '1007_s_at')
# - 'Gene Symbol' column: contains the gene symbols (e.g., 'DDR1 /// MIR4640')

# 3. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print preview of the mapping
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 4. Convert probe-level measurements to gene expression data
# This distributes probe expression values evenly across genes and then sums by gene
try:
    gene_data_mapped = apply_gene_mapping(gene_data, gene_mapping)
    print(f"Successfully mapped gene expression data: {gene_data_mapped.shape[0]} genes × {gene_data_mapped.shape[1]} samples")
    
    # 5. Normalize gene symbols 
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data_mapped)
    print(f"After normalization: {gene_data_normalized.shape[0]} unique gene symbols")
    
    # 6. Print information about the gene expression data
    print("\nGene expression data after mapping:")
    print(f"Number of genes: {len(gene_data_normalized)}")
    print("Preview of the first few genes:")
    print(gene_data_normalized.head())
    
    # 7. Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_normalized.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to: {out_gene_data_file}")
    
    # Store the final processed gene data
    gene_data = gene_data_normalized
    
except Exception as e:
    print(f"Error during gene mapping: {e}")
    raise


Extracted raw gene expression data: 54675 probes × 92 samples


Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}
Successfully mapped gene expression data: 21278 genes × 92 samples


After normalization: 19845 unique gene symbols

Gene expression data after mapping:
Number of genes: 19845
Preview of the first few genes:
          GSM3759992  GSM3759993  GSM3759994  GSM3759995  GSM3759996  \
Gene                                                                   
A1BG        4.390919    9.637094    5.370776    7.376019    9.747455   
A1BG-AS1    4.498580    4.911001    4.409248    4.958840    4.126732   
A1CF        7.712909   17.768014    8.704946   14.905013   16.923252   
A2M        14.491904   16.222561   15.166473   15.598188   15.317525   
A2M-AS1     6.186831    4.286041    5.067774    5.807062    3.963854   

          GSM3759997  GSM3759998  GSM3759999  GSM3760000  GSM3760001  ...  \
Gene                                                                  ...   
A1BG        7.568074   12.627785   12.227179    7.042437    5.118175  ...   
A1BG-AS1    5.894118    4.571268    4.925717    4.390274    4.578439  ...   
A1CF        7.351392   21.828093   20.830584   1

Gene expression data saved to: ../../output/preprocess/Mesothelioma/gene_data/GSE131027.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Create clinical data from the sample IDs in the gene expression data
# Since all samples are from tumor tissue, they all have mesothelioma (but this is not useful as a trait)
sample_ids = gene_data.columns.tolist()
print(f"Number of samples: {len(sample_ids)}")

# Extract gender information from the original matrix file
gender_data = {}
try:
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Read the matrix file to extract sample characteristics
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Display unique values in each row of clinical data
    characteristics_dict = get_unique_values_by_row(clinical_data)
    print("Sample characteristics dictionary:")
    print(characteristics_dict)
    
    # Try to find gender information in the characteristics
    gender_row = None
    for idx, values in characteristics_dict.items():
        if any('sex:' in str(v).lower() for v in values):
            gender_row = idx
            break
    
    if gender_row is not None:
        # Extract gender data from the row
        for col in clinical_data.columns:
            if col != '!Sample_geo_accession':
                continue
                
            for idx, row in clinical_data.iterrows():
                if idx == gender_row:
                    for i, sample_id in enumerate(clinical_data.iloc[0].values):
                        if i > 0 and sample_id in sample_ids:  # Skip the first column (header)
                            gender_val = clinical_data.iloc[idx, i]
                            if isinstance(gender_val, str) and 'sex:' in gender_val.lower():
                                gender = 0 if 'f' in gender_val.lower() else 1 if 'm' in gender_val.lower() else None
                                gender_data[sample_id] = gender
except Exception as e:
    print(f"Error extracting gender data: {e}")

# Create a clinical dataframe
clinical_df = pd.DataFrame(index=sample_ids)
clinical_df['Mesothelioma'] = 1  # All samples have mesothelioma

# Add gender if available
if gender_data:
    clinical_df['Gender'] = clinical_df.index.map(lambda x: gender_data.get(x))
    print(f"Added gender data for {sum(pd.notna(clinical_df['Gender']))} samples")

print("Clinical data preview:")
print(clinical_df.head())

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data (transpose gene expression data to have samples as rows)
linked_data = pd.concat([clinical_df, gene_data_normalized.T], axis=1)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, 'Mesothelioma')
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait is biased (it will be since all samples are mesothelioma)
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Mesothelioma')

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,  # We do have trait data, it's just that all values are the same
    is_biased=is_trait_biased,  # This will be True since all samples have the same trait value
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from mesothelioma patients only (no controls), making trait biased."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to biased trait variable. Final linked data not saved.")

Shape of gene data after normalization: (19845, 92)


Saved normalized gene data to ../../output/preprocess/Mesothelioma/gene_data/GSE131027.csv
Number of samples: 92


Sample characteristics dictionary:
{0: ['tissue: tumor biopsy'], 1: ['cancer: Breast cancer', 'cancer: Colorectal cancer', 'cancer: Bile duct cancer', 'cancer: Mesothelioma', 'cancer: Urothelial cancer', 'cancer: Pancreatic cancer', 'cancer: Melanoma', 'cancer: Hepatocellular carcinoma', 'cancer: Ovarian cancer', 'cancer: Cervical cancer', 'cancer: Head and Neck cancer', 'cancer: Sarcoma', 'cancer: Prostate cancer', 'cancer: Adenoid cystic carcinoma', 'cancer: NSCLC', 'cancer: Oesophageal cancer', 'cancer: Thymoma', 'cancer: Others', 'cancer: CUP', 'cancer: Renal cell carcinoma', 'cancer: Gastric cancer', 'cancer: Neuroendocrine cancer', 'cancer: vulvovaginal'], 2: ['mutated gene: ATR', 'mutated gene: FAN1', 'mutated gene: ERCC3', 'mutated gene: FANCD2', 'mutated gene: BAP1', 'mutated gene: DDB2', 'mutated gene: TP53', 'mutated gene: ATM', 'mutated gene: CHEK1', 'mutated gene: BRCA1', 'mutated gene: WRN', 'mutated gene: CHEK2', 'mutated gene: BRCA2', 'mutated gene: XPC', 'mutated gene:

Shape of linked data after handling missing values: (92, 19846)
Quartiles for 'Mesothelioma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Mesothelioma' in this dataset is severely biased.

Dataset validation failed due to biased trait variable. Final linked data not saved.
